In [7]:
import pandas as pd
import numpy as np
def get_mut(row):
    try:
        wt_aa = row["clean_mut_wt"][0]
        mut_aa = row["clean_mut_wt"][-1]
        pos = int(row["clean_mut_wt"][1:-1])
        seq = list(row["Sequence"])
        assert len(seq) > pos
        assert seq[pos-1] == wt_aa
        seq[pos-1] = mut_aa
        return "".join(seq)
    except AssertionError:
        return None
df = pd.read_csv("dataset/intenzy.csv")
df = df[df["substrate_kinetics"].apply(lambda x: ";" not in x)].reset_index(drop=True)
df["val"] = np.log10((df["kcat_mut"]/df["Km_mut"])/(df["kcat_wt"]/df["Km_wt"]))
df["p_kcat"] = np.log10((df["kcat_mut"]/df["kcat_wt"]))
df["p_km"] = np.log10((df["Km_mut"]/df["Km_wt"]))
seqs = pd.read_csv("dataset/idmapping.tsv", sep="\t")[["From", "Entry", "Sequence"]]
seqs.dropna(inplace=True)
df = pd.merge(df, seqs, left_on="uniprot", right_on="From", how="inner")
df.drop(columns=["From", "Entry"], inplace=True)
df["mut_seq"] = df.apply(get_mut, axis=1)
df.dropna(subset=["mut_seq"], inplace=True)
smiles_df = pd.read_csv("dataset/smiles.smi")
cleaned_df = pd.merge(df, smiles_df, left_on="substrate_kinetics", right_on="name")[["_id", "Sequence", "mut_seq", "smiles", "val"]]
cleaned_df.columns = ["id", "wt_seqs", "mut_seqs", "smiles", "val"]
cleaned_df.to_csv("dataset/train_data_reg.csv", index=False)
cleaned_df_reg = cleaned_df.copy()
cleaned_df_reg["val"] = cleaned_df_reg["val"].apply(lambda x: int(x > 0))
cleaned_df_reg.to_csv("dataset/train_data_class.csv", index=False)

In [8]:
cleaned_df.head()

,id,wt_seqs,mut_seqs,smiles,val
0,63262552b980af2249bd882b,MRAVRLVEIGKPLSLQEIGVPKPKGPQVLIKVEAAGVCHSDVHMRQ...,MRAVRLVEIGKPLSLQEIGVPKPKGPQVLIKVEAAGVCHSDVHMRQ...,CCCCCCCO,-1.238986
1,63262552b980af2249bd882f,MRAVRLVEIGKPLSLQEIGVPKPKGPQVLIKVEAAGVCHSDVHMRQ...,MRAVRLVEIGKPLSLQEIGVPKPKGPQVLIKVEAAGVCHSDVHMRQ...,C1=CC=C(C=C1)CO,-0.676442
2,63262552b980af2249bd8836,MKAAVVEQFKEPLKIKEVEKPTISYGEVLVRIKACGVCHTDLHAAH...,MKAAVVEQFKEPLKIKEVEKPTISAGEVLVRIKACGVCHTDLHAAH...,C1=CC=C(C=C1)CO,-0.268555
3,63262552b980af2249bd8835,MKAAVVEQFKEPLKIKEVEKPTISYGEVLVRIKACGVCHTDLHAAH...,MKAAVVEQFKEPLKIKEVEKPTISYGEVLVRIKACGVCHTDLHAAH...,C1=CC=C(C=C1)CO,0.006921
4,63262552b980af2249bd8830,MRAVRLVEIGKPLSLQEIGVPKPKGPQVLIKVEAAGVCHSDVHMRQ...,MRAVRLVEIGKPLSLQEIGVPKPKGPQVLIKVEAAGVCHSDVHMRQ...,CC(C)C=O,-1.652125


In [9]:
cleaned_df_reg.head()

,id,wt_seqs,mut_seqs,smiles,val
0,63262552b980af2249bd882b,MRAVRLVEIGKPLSLQEIGVPKPKGPQVLIKVEAAGVCHSDVHMRQ...,MRAVRLVEIGKPLSLQEIGVPKPKGPQVLIKVEAAGVCHSDVHMRQ...,CCCCCCCO,0
1,63262552b980af2249bd882f,MRAVRLVEIGKPLSLQEIGVPKPKGPQVLIKVEAAGVCHSDVHMRQ...,MRAVRLVEIGKPLSLQEIGVPKPKGPQVLIKVEAAGVCHSDVHMRQ...,C1=CC=C(C=C1)CO,0
2,63262552b980af2249bd8836,MKAAVVEQFKEPLKIKEVEKPTISYGEVLVRIKACGVCHTDLHAAH...,MKAAVVEQFKEPLKIKEVEKPTISAGEVLVRIKACGVCHTDLHAAH...,C1=CC=C(C=C1)CO,0
3,63262552b980af2249bd8835,MKAAVVEQFKEPLKIKEVEKPTISYGEVLVRIKACGVCHTDLHAAH...,MKAAVVEQFKEPLKIKEVEKPTISYGEVLVRIKACGVCHTDLHAAH...,C1=CC=C(C=C1)CO,1
4,63262552b980af2249bd8830,MRAVRLVEIGKPLSLQEIGVPKPKGPQVLIKVEAAGVCHSDVHMRQ...,MRAVRLVEIGKPLSLQEIGVPKPKGPQVLIKVEAAGVCHSDVHMRQ...,CC(C)C=O,0
